In [7]:
###############################################################################################
#
#                            Real Time Plot Graphical User Interface
#                                  
################################################################################################

# Works in connection with the AnalogReadSerial_firm_II

#======================= Loading of Packages =================================#

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
import numpy as np
import serial as ser # pip install pyserial
import time

# ======================== Global Variables =======================================#

data = np.array([])
cond = False

# ===================== Serial Communication Port =================================#

serial_port = ser.Serial('COM5',9600);
serial_port.reset_input_buffer()

# ========================= Plot data ========================================#

def plot_data():
    global cond, data
    
    if (cond == True):
        
        a = serial_port.readline() # reading serial data
        a.decode() # decoding seria data
        
        if(len(data) < 100):
            data = np.append(data, float(a[0:4]))
        else:
            data[0:99] = data[1:100]
            data[99] = float(a[0:4])
        
        # data_sp = np.append(data_sp, sp)
        # ax.clear
        # ax.plot(data)
        lines.set_xdata(np.arange(0,len(data)))
        lines.set_ydata(data)
        
        canvas.draw()
        
    root.after(1, plot_data)
        
def plot_start():
    global cond
    cond = True
    serial_port.reset_input_buffer
    
def plot_stop():
    global cond
    cond = False

#========================= Main GUI code =====================================#

root = tk.Tk()
root.title('Real Time Plot')
root.configure(background = 'light blue')
root.geometry("900x600") # set the window size

# Hasta aqui si solo le agrego "root.mainloop()" obtengo una ventanita con fondo azul como fue configurado 

#==================== Create Plot Object on GUI ==============================#

# Add figure canvas 

fig = Figure();
ax = fig.add_subplot(111)

# ax = plt.axes(xlim = (0,100), ylim = (0,120)); # displaying only 100 sam

ax.set_title('Serial Data'); # Could be for example motor speed
ax.set_xlabel('Sample')
ax.set_ylabel('Voltage') # Could be for example Speed (RPM)
ax.set_xlim(0,100)
ax.set_ylim(-0.5,6)
lines = ax.plot([],[])[0]

canvas = FigureCanvasTkAgg(fig, master = root) # A tk.drawingArea.
canvas.get_tk_widget().place(x = 10, y= 10, width = 500, height = 400)
canvas.draw()

######################### Hasta aqui incluyendo "root.mainloop()", optenemos la deseada ventanita con con el espacio para poner el plot

# ============================ Create Buttons ======================================== #

root.update();
start = tk.Button(root, text = 'Start', font = ('calibri', 12), command = lambda: plot_start())
start.place(x = 100, y = 500)

root.update();
stop = tk.Button(root, text = "Stop", font = ('calibri', 12), command = lambda: plot_stop())
stop.place(x = start.winfo_x()+start.winfo_reqwidth() + 20, y = 500)

# =============================== Clossing ========================================== #

root.after(1, plot_data) # sampling frequency of 1 milisecond
root.mainloop() 
serial_port.close()    